In [1]:
import pandas as pd
import numpy as np
import copy, time
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

df_train = pd.read_csv('./train_data.csv')
df_test = pd.read_csv('./test_features.csv')
print(df_train.shape)
print(df_train.columns)
df_train.head()

(113, 22)
Index(['name', 'bonus', 'deferral_payments', 'deferred_income',
       'director_fees', 'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
       'loan_advances', 'long_term_incentive', 'other', 'poi',
       'restricted_stock', 'restricted_stock_deferred', 'salary',
       'shared_receipt_with_poi', 'to_messages', 'total_payments',
       'total_stock_value'],
      dtype='object')


,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [6]:
train_Y = df_train['poi'] #目標欄位
name = df_test['name'] 
df_train = df_train.drop(['name','poi'] , axis=1) 
df_test = df_test.drop(['name'] , axis=1)
df = pd.concat([df_train, df_test],sort=False) #資料一併做前處理
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [8]:
def na_num(df_data):
    num = (df_data.isnull().sum()/len(df)).sort_values(ascending=False)
    return num
na_num(df) #欄位缺失率

loan_advances                0.972603
director_fees                0.883562
restricted_stock_deferred    0.876712
deferral_payments            0.732877
deferred_income              0.664384
long_term_incentive          0.547945
bonus                        0.438356
from_messages                0.410959
from_poi_to_this_person      0.410959
from_this_person_to_poi      0.410959
shared_receipt_with_poi      0.410959
to_messages                  0.410959
other                        0.363014
expenses                     0.349315
salary                       0.349315
exercised_stock_options      0.301370
restricted_stock             0.246575
email_address                0.239726
total_payments               0.143836
total_stock_value            0.136986
dtype: float64

In [ ]:
[ 'director_fees', 'expenses',
       'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
       'loan_advances',  
       'restricted_stock', 'restricted_stock_deferred',
       'shared_receipt_with_poi', 'to_messages', 'total_payments',
       'total_stock_value']

In [ ]:
#缺失值比率過高，先捨去欄位
df = df.drop(['loan_advances'],axis=1)

In [10]:
#類別缺失值填補
none_cols = ['email_address']

for col in none_cols:
    df[col] = df[col].fillna("None")

In [ ]:
#部分欄位缺失值填補0
zero_cols = ['long_term_incentive', 'deferred_income','deferral_payments','director_fees',]

for col in zero_cols:
    df[col] = df[col].fillna(0)

In [ ]:
#部分欄位缺失值填補眾數
mod_cols = [ 'salary','bonus',]

for col in none_cols:
    df[col] = df[col].fillna(f[col].mode)

In [ ]:
#部分欄位缺失值填補平均數
med_cols = ['other','exercised_stock_options',]

for col in none_cols:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
#確認缺失值填補是否完整
na_num(df) 

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
result = next(kf.split(df), None)
train = df.iloc[result[0]]
valid =  df.iloc[result[1]]
df_train = train
df_valid = valid

In [ ]:
# 使用三種模型 : 線性迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
linear = LinearRegression(normalize=False, fit_intercept=True, copy_X=True)
gdbt = GradientBoostingRegressor(tol=0.1, subsample=0.37, n_estimators=200, max_features=20, 
                                 max_depth=6, learning_rate=0.03)
rf = RandomForestRegressor(n_estimators=300, min_samples_split=9, min_samples_leaf=10, 
                           max_features='sqrt', max_depth=8, bootstrap=False)

In [ ]:
sub = pd.DataFrame({'name': name, 'poi': np.expm1(y_pred)})
sub['poi'] = sub['poi'].map(lambda x:True if x>0.5 else False) 
sub.to_csv('EnronFraud.csv', index=False)